In [1]:
import sys; sys.path.append("../../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.lama import TabularLamaUtilized
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-08 13:26:27]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

In [3]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../data/")
CONFIG_PATH = Path("../../../configs/config.yaml")
N_JOBS = 16

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [7]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [8]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [9]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [10]:
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [11]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [12]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [13]:
metric = RocAuc()

In [14]:
model = TabularLamaUtilized(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-08 13:27:24,414] - [   START    ] - Fitting TabularLamaUtilized
[13:27:24] Start automl utilizator with listed constraints:
[13:27:24] - time: 7200.00 seconds
[13:27:24] - CPU: 16 cores
[13:27:24] - memory: 16 GB

[13:27:24] If one preset completes earlier, next preset configuration will be started

[13:27:24] ==================================================
[13:27:24] Start 0 automl preset configuration:
[13:27:24] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[13:27:24] Stdout logging level is INFO.
[13:27:24] Task: binary

[13:27:24] Start automl preset with listed constraints:
[13:27:24] - time: 7200.00 seconds
[13:27:24] - CPU: 16 cores
[13:27:24] - memory: 16 GB

[13:27:24] Train data shape: (413194, 63)

[13:27:35] Layer 1 train process start. Time left 7188.70 secs
[13:27:48] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:28:33] Fitting Lvl_0_Pipe

[14:00:53] Task: binary

[14:00:53] Start automl preset with listed constraints:
[14:00:53] - time: 5190.98 seconds
[14:00:53] - CPU: 16 cores
[14:00:53] - memory: 16 GB

[14:00:53] Train data shape: (413194, 63)

[14:00:54] Layer 1 train process start. Time left 5190.17 secs
[14:01:07] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:01:56] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7878750314025083
[14:01:56] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:01:56] Time left 5128.30 secs

[14:02:07] Selector_LightGBM fitting and predicting completed
[14:02:19] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[14:03:08] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8076809265966576
[14:03:08] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[14:03:08] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs
[14:08:09] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM complet

[14:42:01] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.8121930536698967
[14:42:01] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[14:42:02] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[14:42:54] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.8067263183241322
[14:42:54] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[14:42:54] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 300.00 secs
[14:48:05] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[14:48:05] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[14:49:53] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8075557040558934
[14:49:53] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[14:49:53] Time left 2250.78 secs

[14:49:53] Layer 1 training completed.

[14:49:53] Blending: optimization starts with equal weights and score 0.8097974977616682
[14:49:59] Blending: 

[15:07:10] Blending: no score update. Terminated

[15:07:10] Automl preset training completed in 1019.65 seconds

[15:07:10] Model description:
Final prediction for new objects (level 0) = 
	 0.23267 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.65069 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.11664 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 

[15:07:10] ==================================================
[15:07:10] Start 6 automl preset configuration:
[15:07:10] conf_6_sel_type_1_tuning_full_no_int_lgbm.yml, random state: {'reader_params': {'random_state': 48}, 'nn_params': {'random_state': 48}, 'general_params': {'return_all_predictions': False}}
[15:07:10] Stdout logging level is INFO.
[15:07:10] Task: binary

[15:07:10] Start automl preset with listed constraints:
[15:07:10] - time: 1214.01 seconds
[15:07:10] - CPU: 16 cores
[15:07:10] - memory: 16 GB

[15:07:10] Train data shape: (413194, 63)

[15:07:21] Layer 1 train process start. Time le

In [15]:
MODEL_NAME = "lamau_81425_full_dataset"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [16]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [14]:
model = TabularLamaUtilized(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 30, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 12:21:05,781] - [   START    ] - Fitting TabularLamaUtilized
[12:21:05] Start automl utilizator with listed constraints:
[12:21:05] - time: 3600.00 seconds
[12:21:05] - CPU: 16 cores
[12:21:05] - memory: 16 GB

[12:21:05] If one preset completes earlier, next preset configuration will be started

[12:21:05] ==================================================
[12:21:05] Start 0 automl preset configuration:
[12:21:05] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[12:21:05] Stdout logging level is INFO.
[12:21:05] Task: binary

[12:21:05] Start automl preset with listed constraints:
[12:21:05] - time: 3600.00 seconds
[12:21:05] - CPU: 16 cores
[12:21:05] - memory: 16 GB

[12:21:05] Train data shape: (413194, 63)

[12:21:18] Layer 1 train process start. Time left 3587.51 secs
[12:21:32] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:22:23] Fitting Lvl_0_Pipe

[12:55:40] Task: binary

[12:55:40] Start automl preset with listed constraints:
[12:55:40] - time: 1525.31 seconds
[12:55:40] - CPU: 16 cores
[12:55:40] - memory: 16 GB

[12:55:40] Train data shape: (413194, 63)

[12:55:41] Layer 1 train process start. Time left 1524.43 secs
[12:55:55] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:56:43] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7877730815141979
[12:56:43] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:56:43] Time left 1462.35 secs

[12:56:53] Selector_LightGBM fitting and predicting completed
[12:57:06] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[12:58:00] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8076809265966576
[12:58:00] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[12:58:00] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 135.76 secs
[13:00:37] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM complet

In [15]:
MODEL_NAME = "lamau_814_full_dataset"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [16]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method